# Detecting Change
This will allow both for change detection and the visualization of change

based on
https://developers.google.com/earth-engine/tutorials/community/imad-tutorial-pt1

Multivariate Alteration Detection (MAD)
    Reweighted to create the iMAD with interative reweighing 
    Widely used for change information from visual/infrared imagery 

In [1]:
# earth engine
import ee


In [2]:
ee.Authenticate(auth_mode='karen')


True

In [3]:

ee.Initialize(project='ee-karenpedersen2')

In [4]:
import sys
print(sys.executable)

/home/the-laboratory/Documents/github/GoogleEarthEngineWalkThrough/.venv/bin/python


In [5]:
# modules
import pkg_resources
import geemap
import ipyleaflet as lf
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2
%matplotlib inline

from pprint import pprint  # for pretty printing


/tmp/ipykernel_36694/3252214429.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
# Truncate a 1-D array to dec decimal places
def trunc(values, dec = 3):
    return np.trunc(values*10**dec)/(10**dec)


In [7]:
# Display an image in a one percent linear stretch
def display_ls(image, map, name, centered = False):
    bns = image.bandNames().length().getInfo()
    if bns == 3:
        image = image.rename('B1', 'B2', 'B3')
        pb_99 = ['B1_p99', 'B2_p99', 'B3_p99']
        pb_1 = ['B1_p1', 'B2_p1', 'B3_p1']
        img = ee.Image.rgb(image.select('B1'), image.select('B2'), image.select('B3'))
    else:
        image = image.rename('B1')
        pb_99 = ['B1_p99']
        pb_1 = ['B1_p1']
        img = image.select('B1')
    percentiles = image.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
    mx = percentiles.values(pb_99)
    if centered:
        mn = ee.Array(mx).multiply(-1).toList()
    else:
        mn = percentiles.values(pb_1)
    map.addLayer(img, {'min': mn, 'max': mx}, name)


# Simple differances
Two Images with the same scene and same sensor, at different times.

To see if there are differances we subtract two normalized or calibrated images from each
other.

First I will use the example because they provide a data object I do not really understand yet.

In [8]:
# I do not understand what this is
aoi = ee.FeatureCollection(
    'projects/google/imad_tutorial/landkreis_olpe_aoi').geometry()


Then two sentinel-2 scenes within the area defined above

In [9]:
def collect(aoi, t1a ,t1b, t2a, t2b):
    try:
        im1 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                               .filterBounds(aoi)
                               .filterDate(ee.Date(t1a), ee.Date(t1b))
                               .filter(ee.Filter.contains(rightValue=aoi,leftField='.geo'))
                               .sort('CLOUDY_PIXEL_PERCENTAGE')
                               .first()
                               .clip(aoi) )
        im2 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                               .filterBounds(aoi)
                               .filterDate(ee.Date(t2a), ee.Date(t2b))
                               .filter(ee.Filter.contains(rightValue=aoi,leftField='.geo'))
                               .sort('CLOUDY_PIXEL_PERCENTAGE')
                               .first()
                               .clip(aoi) )
        timestamp = im1.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        timestamp = im2.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        return (im1, im2)
    except Exception as e:
        print('Error: %s'%e)

im1, im2 = collect(aoi, '2021-06-01', '2021-06-30', '2022-06-01', '2022-06-30')

Sun Jun 13 10:36:49 2021
Thu Jun 16 10:46:56 2022


In [10]:
# Interactive map for viewing images
# does not display very buggy in linux anf vscode
M1 = geemap.Map()
M1.centerObject(aoi, 11)

M1

Map(center=[51.08617675675536, 7.976478425048881], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
visirbands = ['B2','B3','B4','B8','B11','B12']
visbands = ['B2','B3','B4']

diff = im1.subtract(im2).select(visbands)
display_ls(im1.select(visbands), M1, 'Image 1')
display_ls(im2.select(visbands), M1, 'Image 2')
display_ls(diff, M1, 'Difference')


In [12]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)